# Finetuning a Convolutional Neural Network

In this exercise, you will have to finetune a pretrained CNN model on the CIFAR10 dataset. The data loading and model testing logic are already included in your code. You will have to create the model and the training loop.

**In this workspace you have GPU to help train the model but it is best practice to DISABLE it while writing code and only ENABLE it when you are training.** 

Here are the steps you need to do to complete this exercise:

1. Finish the `create_model()` function. You should use a pretrained model. You are free to choose any pre-trained model that you want to use. 
2. Finish the `train()` function. This function should validate the accuracy of the model during the training stage. You should stop the training when this validation accuracy stops increasing.
3. Save all your work and then **ENABLE** the GPU
4. Run the file to make sure that the model is training properly.
5. If it works, remember to **DISABLE** the GPU before moving to the next page. 

In case you get stuck, you can look at the solution by clicking the jupyter symbol at the top left and navigating to `finetune_a_cnn_solution.py`.

## Try It Out!
- See how your accuracy changes when using other pre-trained models.
- Play around with the number of layers and neurons in your model. How does the accuracy change? How long does it take to train the model?
- Can you create the same network in TensorFlow as well?

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time

def test(model, test_loader, criterion):
    model.eval()
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    total_loss = running_loss / len(test_loader)
    total_acc = running_corrects / len(test_loader)
    
def train(model, train_loader, validation_loader, criterion, optimizer):
    epochs=50
    best_loss=1e6
    image_dataset={'train':train_loader, 'valid':validation_loader}
    loss_counter=0
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Running on Device {device}")

    
    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase=='train':
                model.train()
                print("model.train()")
            else:
                model.eval()
            #TODO: Finish the rest of the training code
            # The code should stop training when the validation accuracy
            # stops increasing
            running_loss = 0.0
            running_corrects = 0
            running_samples=0

            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                print(f"step {step}")
                inputs=inputs.to(device)
                labels=labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase=='train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                running_samples+=len(inputs)
                if running_samples % 2000  == 0:
                    accuracy = running_corrects/running_samples
                    print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%) Time: {}".format(
                            running_samples,
                            len(image_dataset[phase].dataset),
                            100.0 * (running_samples / len(image_dataset[phase].dataset)),
                            loss.item(),
                            running_corrects,
                            running_samples,
                            100.0*accuracy,
                            time.asctime() # for measuring time for testing, remove for students and in the formatting
                        )
                    )
                    
                    #NOTE: Comment lines below to train and test on whole dataset
                    #if running_samples>(0.2*len(image_dataset[phase].dataset)):
                    #    break

            epoch_loss = running_loss / running_samples
            epoch_acc = running_corrects / running_samples
        
            if phase=='valid':
                if epoch_loss<best_loss:
                    best_loss=epoch_loss
                else:
                    loss_counter+=1

        if loss_counter==1:
            break
    return model

def create_model():
    model = models.resnet18(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False   

    num_features=model.fc.in_features
    model.fc = nn.Sequential(
                   nn.Linear(num_features, 10))
    return model

batch_size=32
training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

testing_transform = transforms.Compose([transforms.ToTensor(),
    transforms.RandomResizedCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)

model=create_model()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

train(model, trainloader, testloader, criterion, optimizer)

test(model, test_loader, criterion)

Files already downloaded and verified
Files already downloaded and verified
Running on Device cpu
Epoch 0, Phase train
model.train()
step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99
step 100
step 101
step 102
step 103
step 104
step 105
step 106
step 107
step 

KeyboardInterrupt: 